In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

import sympy as sp # Symbolic Python
import numpy as np # Arrays, matrices and corresponding mathematical operations
from IPython.display import Latex, display, Markdown # For displaying Markdown and LaTeX code
from ipywidgets import widgets # Interactivity module

# Function for the conversion of array/matrix to LaTeX/Markdown format.
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)

In order to check the stability of the system, please define the characteristic polynomial by inserting its order and corresponding coefficients.

In [2]:
polynomialOrder = input ("Insert the order of the characteristic polynomial:")
try:
    val = int(polynomialOrder)
except ValueError:
    display(Markdown('Order of the polynomial has to be an integer. Please re-enter the value.+'))
display(Markdown('Please insert the coefficients of the characteristic polynomial. Use K for undefined coefficient.'))
text=[None]*(int(polynomialOrder)+1)
for i in range(int(polynomialOrder)+1):
    text[i]=widgets.Text(description=('s%i'%(-(i-int(polynomialOrder)))))
    display(text[i])
btn=widgets.Button(description="Confirm")
display(btn)
coef=[None]*(int(polynomialOrder)+1)
def on_button_clicked(btn):
    for i in range(int(polynomialOrder)+1):
        if text[i].value=='' or text[i].value=='Please insert a coefficient':
            text[i].value='Please insert a coefficient'
        else:
            try:
                coef[i]=float(text[i].value)
            except ValueError:
                if text[i].value!='' or text[i].value!='Please insert a coefficient':
                    coef[i]=sp.var(text[i].value)
btn.on_click(on_button_clicked)

Insert the order of the characteristic polynomial:2


Please insert the coefficients of the characteristic polynomial. Use K for undefined coefficient.

Text(value='', description='s2')

Text(value='', description='s1')

Text(value='', description='s0')

Button(description='Confirm', style=ButtonStyle())

In [3]:
coef.reverse()
enacba="$"
for i in range (int(polynomialOrder),-1,-1):
    if i==int(polynomialOrder):
        enacba=enacba+str(coef[i])+"s^"+str(i)
    elif i==1:
        enacba=enacba+"+"+str(coef[i])+"s"
    elif i==0:
        enacba=enacba+"+"+str(coef[i])+"$"
    else:
        enacba=enacba+"+"+str(coef[i])+"s^"+str(i)
coef.reverse()
display(Markdown('The characteristic polynomial of interest is:'), Markdown(enacba))

The characteristic polynomial of interest is:

$1.0s^2+Ks+2.0$

Would you like to use Routh or Hurwitz criterion to check the stability?

In [4]:
w=widgets.Select(
    options=['Please select...','Routh', 'Hurwitz'],
    value='Please select...',
    rows=3,
    description='Select:',
    disabled=False
)
display(w)

Select(description='Select:', options=('Please select...', 'Routh', 'Hurwitz'), rows=3, value='Please select..…

In [7]:
K=sp.Symbol('K');
# Check if selected stability criterion is Routh or Hurwitz.
if w.value=='Please select...':
    raise ValueError('Please select the Routh or Hurwitz stability criterion.')

elif w.value=='Routh':
    
    s=np.zeros([len(coef),3],dtype=object)
    for i in range(3):
        try:
            s[0,i]=coef[i*2]
            s[1,i]=coef[i*2+1]
        except IndexError:
            s[0,i]=0
            s[1,i]=0
    for i in range(len(coef)-2):
        for j in range(2):
            if j==0:
                s[i+2,j]=(s[i+1,j]*s[i,j+1]-s[i,j]*s[i+1,j+1])/s[i+1,j]
            elif j==1:
                s[i+2,j]=(s[i+1,j-1]*s[i,j+1]-s[i,j-1]*s[i+1,j+1])/s[i+1,j-1]
                
    if all(isinstance(x, (int,float)) for x in coef):
        positive_check=s[:,0]>0
        negative_check=s[:,0]<0
        if all(positive_check)==True:
            display(Markdown('System is stable, because all the elements in the first column of the Routh table are positive.'))
        elif all(negative_check)==True:
            display(Markdown('System is stable, because all the elements in the first column of the Routh table are negative.'))
        else:
            display(Markdown('System is unstable, because the elements in the first column of the Routh table do not have the same sign.'))
        display(Markdown('Routh table $%s$' % vmatrix(s)))
    else:
        testSign=[]
        for i in range(len(s)):
            if isinstance(s[i,0],(int,float)):
                testSign.append(s[i,0]>0)
        solution=[]
        if all(elem == True for elem in testSign):
            for x in s[:,0]:
                if not isinstance(x,(sp.numbers.Integer,sp.numbers.Float,int,float)):
                    solution.append(sp.solve(x>0,K)) # Define the solution for each value of the determinant
            display(Markdown('Routh table $%s$' % vmatrix(s)))
            display(Markdown('All the known coefficients in the first column of Routh table are positive, therefore the system is stable for:'))
            print(solution)            
        elif all(elem == False for elem in test):
            for x in s[:,0]:
                if not isinstance(x,(sp.numbers.Integer,sp.numbers.Float,int,float)):
                    solution.append(sp.solve(x<0,K)) # Define the solution for each value of the determinant
            display(Markdown('Routh table $%s$' % vmatrix(s)))
            display(Markdown('All the known coefficients in the first column of Routh table are negative, therefore the system is stable for:'))
            print(solution)
        else:
            display(Markdown('Routh table $%s$' % vmatrix(s)))
            display(Markdown('System is unstable, beacuse the signs of the coefficients in the first column differ between each other.'))
        
        

elif w.value=='Hurwitz':
    
    # Check if all the coefficients are numbers or not and preallocate basic determinant.

    if all(isinstance(x, (int,float)) for x in coef):
        determinant=np.zeros([len(coef)-1,len(coef)-1])
    else:
        determinant=np.zeros([len(coef)-1,len(coef)-1],dtype=object)

    # Define the first two rows of the basic determinant.    
    for i in range(len(coef)-1):
        try:
            determinant[0,i]=coef[2*i+1]
            determinant[1,i]=coef[2*i]
        except:
            determinant[0,i]=0
            determinant[1,i]=0
    # Define the remaining rows of the basic determinant by shifting the first two rows.        
    for i in range(2,len(coef)-1):
        determinant[i,:]=np.roll(determinant[i-2,:],1)
        determinant[2:,0]=0

    # Define all the subdeterminants.
    subdet=[];
    for i in range(len(determinant)-1):
        subdet.append(determinant[0:i+1,0:i+1])

    # Append the basic determinant to the subdeterminants' array.
    subdet.append(determinant)

    # Check if all coefficients are numbers.
    if all(isinstance(x, (int,float)) for x in coef):
        det_value=[] # Preallocate array containing values of all determinants.
        for i in range(len(subdet)):
            det_value.append(np.linalg.det(subdet[i])); # Calculate determinant and append the values to det_value.

        if all(i > 0 for i in det_value)==True: # Check if all values in det_value are positive or not.
            display(Markdown('System is stable, because all the determinants are positive.'))
            for i in range(len(subdet)):
                display(Markdown('$\Delta_{%i}=$'%(i+1) + '$%s$' %vmatrix(subdet[i]) + '$=%s$' %det_value[i]))
        else:
            display(Markdown('System is unstable, because all the determinants are not positive.'))
            for i in range(len(subdet)):
                display(Markdown('$\Delta_{%i}=$'%(i+1) + '$%s$' %vmatrix(subdet[i]) + '$=%s$' %det_value[i]))
    else:
        subdetSym=[] # Preallocate subdetSym.
        det_value=[] # Preallocate det_value.
        solution=[] # Preallocate solution.
        for i in subdet:
            subdetSym.append(sp.Matrix(i)) # Transform matrix subdet to symbolic.
        for i in range(len(subdetSym)):
            det_value.append(subdetSym[i].det()) # Calculate the value of the determinant.
        testSign=[]
        for i in range(len(det_value)):
            if isinstance(s[i,0],(int,float,sp.numbers.Integer,sp.numbers.Float)):
                testSign.append(s[i,0]>0)
        if all(elem == True for elem in testSign):
            solution=[]
            for x in det_value:
                if not isinstance(x,(sp.numbers.Integer,sp.numbers.Float,int,float)):
                    solution.append(sp.solve(x>0,K)) # Define the solution for each value of the determinant
            for i in range(len(subdet)):
                display(Markdown('$\Delta_{%i}=$'%(i+1) + '$%s$' %vmatrix(subdet[i]) + '$=%s$' %det_value[i]))
            display(Markdown('System is stable, for:'))
            print(solution)    
            
        else:
            display(Markdown('System is unstable, because all the determinants are not positive.'))
            for i in range(len(subdet)):
                display(Markdown('$\Delta_{%i}=$'%(i+1) + '$%s$' %vmatrix(subdet[i]) + '$=%s$' %det_value[i]))
        
        

$\Delta_{1}=$$\begin{vmatrix}
  K\\
\end{vmatrix}$$=K$

$\Delta_{2}=$$\begin{vmatrix}
  K & 0\\
  1.0 & 0\\
\end{vmatrix}$$=0$

System is stable, for:

[(0 < K) & (K < oo)]
